In [9]:
from functions import *

In [10]:
# Read the CSV file into a DataFrame
result_dir = "../../exp_results"

In [11]:
summary_path = result_dir + "/result_summary.csv"
output_path = result_dir + "/correctness_per_dataset.csv"

check_correntness(summary_path, output_path)

    numTasks  utilization  TP  TN  FP  FN  accuracy
0        3.0          0.2  50   0   0   0      1.00
1        3.0          0.4  45   4   0   1      0.98
2        3.0          0.6  44   6   0   0      1.00
3        6.0          0.2  49   1   0   0      1.00
4        6.0          0.4  41   6   0   3      0.94
5        6.0          0.6  36  11   0   3      0.94
6        9.0          0.2  48   0   0   2      0.96
7        9.0          0.4  33   9   0   8      0.84
8        9.0          0.6  15  25   0  10      0.80
9       12.0          0.2  42   0   0   8      0.84
10      12.0          0.4  24  13   0  13      0.74
11      12.0          0.6   7  36   0   7      0.86


In [12]:
# Read the CSV file into a DataFrame
result_dir = "../../exp_results"
summary_path = result_dir + "/result_summary.csv"

# check conservativeness of results for each task
detail_result_dir = result_dir + "/detail_result"
combine_detail_result_path = detail_result_dir + "/combined_detail_result.csv"
combine_detail_result(summary_path,  detail_result_dir, combine_detail_result_path)

output_path = result_dir + "/correctness_per_task.csv"

check_correntness(combine_detail_result_path, output_path)

    numTasks  utilization   TP  TN  FP  FN  accuracy
0        3.0          0.2  150   0   0   0  1.000000
1        3.0          0.4  137   0   0   1  0.992754
2        3.0          0.6  132   0   0   0  1.000000
3        6.0          0.2  294   0   0   0  1.000000
4        6.0          0.4  261   0   0   3  0.988636
5        6.0          0.6  231   0   0   3  0.987179
6        9.0          0.2  448   0   0   2  0.995556
7        9.0          0.4  359   0   0  10  0.972900
8        9.0          0.6  214   0   0  11  0.951111
9       12.0          0.2  592   0   0   8  0.986667
10      12.0          0.4  428   0   0  16  0.963964
11      12.0          0.6  159   0   0   9  0.946429


In [13]:
import pandas as pd

combined_detail_df = pd.read_csv(combine_detail_result_path)
# filter out the tasksets that are not schedulable by the simulator 
TN_df = combined_detail_df[combined_detail_df["simulator_schedulability"] == False]
TN_df

,numCores,numTasks,utilization,tasksetIndex,name,deadline,WCRT(sim),simulator_schedulability,WCRT(prop),proposed_schedulability


In [14]:
summary_df = pd.read_csv(summary_path)
num_cores = 1
num_tasks = 6
utilization = 0.2
taskset_index = 19

# filter out the summary of the taskset with num_cores, num_tasks, utilization, taskset_index
filtered_summary = summary_df[(summary_df["numCores"] == num_cores ) & (summary_df["numTasks"] == num_tasks) & (summary_df["utilization"] == utilization) & (summary_df["tasksetIndex"] == taskset_index)]
filtered_summary

,numCores,numTasks,utilization,tasksetIndex,simulator_schedulability,simulator_timeConsumption(us),proposed_schedulability,proposed_timeConsumption(us)
169,1,6,0.2,19,True,100733,True,337


In [15]:
summary_df = pd.read_csv(summary_path)
filtered_summary = summary_df[(summary_df["simulator_schedulability"] == True) & (summary_df["proposed_schedulability"] == False) &  (summary_df["numTasks"] == 6)]
filtered_summary

,numCores,numTasks,utilization,tasksetIndex,simulator_schedulability,simulator_timeConsumption(us),proposed_schedulability,proposed_timeConsumption(us)
215,1,6,0.4,15,True,121108,False,374
227,1,6,0.4,27,True,110807,False,383
233,1,6,0.4,33,True,130529,False,350
260,1,6,0.6,10,True,176216,False,532
269,1,6,0.6,19,True,214276,False,399
273,1,6,0.6,23,True,132175,False,392


In [16]:
num_cores = 1
num_tasks = 6
utilization = 0.2
taskset_index = 19
detail_result_dir = "../../exp_results/detail_result"

detail_df = get_detail_result(detail_result_dir, num_cores, num_tasks, utilization, taskset_index)
detail_df

,1,2,3,4,5,6
0,,,,,,
id,deadline,name,WCRT_by_simulator,simulator_schedulability,WCRT_by_proposed,proposed_schedulability
1,800000,task1,51416,true,54403,true
2,400000,task2,104541,true,104541,true
3,400000,task3,100802,true,104300,true
4,900000,task4,83837,true,104418,true
5,300000,task5,19774,true,82662,true
6,400000,task6,100711,true,104541,true
